!pip install obspy


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
from collections import defaultdict
import hvsrpy
from hvsrpy import sesame
from io import StringIO
import contextlib

def safe_print(s):
    try:
        print(s)
    except UnicodeEncodeError:
        print(s.encode('utf-8', errors='replace').decode('utf-8'))


plt.style.use(hvsrpy.HVSRPY_MPL_STYLE)

# ✅ 데이터 폴더 지정
data_dir = pathlib.Path("C:/SOLODATA/zonghap/3month/3월부터얻은데이터목록")
sac_files = sorted(data_dir.glob("*.sac"))

# ✅ Z/N/E 그룹화
grouped_files = defaultdict(dict)
for file in sac_files:
    parts = file.stem.split(".")
    if len(parts) < 9:
        continue
    base_id = ".".join(parts[:-1])
    comp = parts[-1]
    grouped_files[base_id][comp] = str(file)

fname_sets = []
for base, comps in grouped_files.items():
    if all(k in comps for k in ["E", "N", "Z"]):
        fname_sets.append([comps["Z"], comps["N"], comps["E"]])

# ✅ 저장 폴더
save_root = pathlib.Path("C:/SOLODATA/zonghap/hvsr_output")
output_dir = save_root / data_dir.name
output_dir.mkdir(parents=True, exist_ok=True)

# ✅ 설정
pre = hvsrpy.settings.HvsrPreProcessingSettings()
pre.detrend = "linear"
pre.window_length_in_seconds = 30
pre.orient_to_degrees_from_north = 0.0
pre.filter_corner_frequencies_in_hz = (0.1, 20.0)

proc = hvsrpy.settings.HvsrTraditionalProcessingSettings()
proc.window_type_and_width = ("tukey", 0.2)
proc.smoothing = dict(operator="konno_and_ohmachi", bandwidth=40,
                      center_frequencies_in_hz=np.geomspace(0.2, 50, 200))
proc.method_to_combine_horizontals = "geometric_mean"
proc.handle_dissimilar_time_steps_by = "frequency_domain_resampling"

# ✅ 루프 시작
for fname_set in fname_sets:
    base_name = pathlib.Path(fname_set[0]).stem.replace(".Z", "")
    print(f"\n▶ Processing: {base_name}")

    try:
        srecords = hvsrpy.read([fname_set])
        srecords = hvsrpy.preprocess(srecords, pre)
        hvsr = hvsrpy.process(srecords, proc)

        # ✅ 피크 탐색
        hvsr.update_peaks_bounded(search_range_in_hz=(0.1, 20))

        # ✅ 피크 추정 (mean_curve 기반)
        frequency = hvsr.frequency
        mean_curve = hvsr.mean_curve(distribution="lognormal")
        peak_idx = np.argmax(mean_curve)
        fn = frequency[peak_idx]
        An = mean_curve[peak_idx]

        fn_std_log = hvsr.std_fn_frequency(distribution="lognormal")
        An_std_log = hvsr.std_fn_amplitude(distribution="lognormal")

        fn_lo = fn / np.exp(fn_std_log)
        fn_hi = fn * np.exp(fn_std_log)

        Tn = 1.0 / fn
        Tn_lo = 1.0 / fn_hi
        Tn_hi = 1.0 / fn_lo

        An_lo = An / np.exp(An_std_log)
        An_hi = An * np.exp(An_std_log)

                # ✅ SESAME 평가 및 통계 요약 저장 (TXT)
        txt_path = output_dir / f"{base_name}_full_summary.txt"
        with open(txt_path, "w", encoding="utf-8") as f:
            buf = StringIO()
            with contextlib.redirect_stdout(buf):
                print("\nSESAME (2004) Clarity and Reliability Criteria:")
                print("-" * 47)
                sesame.reliability(
                    windowlength=pre.window_length_in_seconds,
                    passing_window_count=np.sum(hvsr.valid_window_boolean_mask),
                    frequency=hvsr.frequency,
                    mean_curve=mean_curve,
                    std_curve=hvsr.std_curve(distribution="lognormal"),
                    search_range_in_hz=(None, None),
                    verbose=1
                )
                sesame.clarity(
                    frequency=hvsr.frequency,
                    mean_curve=mean_curve,
                    std_curve=hvsr.std_curve(distribution="lognormal"),
                    fn_std=hvsr.std_fn_frequency(distribution="normal"),
                    search_range_in_hz=(None, None),
                    verbose=1
                )
                print("\nStatistical Summary:")
                print("-" * 20)
                hvsrpy.summarize_hvsr_statistics(hvsr)
                # ✅ 피크 정보 요약 문장도 이 안에 출력
                print(f"\nThe peak of the mean curve is at {fn:.3f} Hz with amplitude {An:.3f}.\n")
                # ✅ 여기! TXT에 포함되도록 print로 출력
                print(f"\nThe peak of the mean curve is at {fn:.3f} Hz with amplitude {An:.3f}.\n")
            f.write(buf.getvalue())  # 모든 내용 한꺼번에 파일에 저장

        # ✅ 요약 표 저장 (CSV)
        rounded_fn= round(fn,3)
        summary_df = pd.DataFrame({
            "Statistic": [
                "Resonant Site Frequency, fn (Hz)",
                "Resonant Site Period, Tn (s)",
                "Resonance Amplitude, An",
                "Peak of the mean curve"
            ],
            "Exponentiated Lognormal Median (units)": [fn, Tn, An,rounded_fn],
            "Lognormal Standard Deviation (log units)": [fn_std_log, fn_std_log, An_std_log,""],
            "-1 Lognormal Standard Deviation (units)": [fn_lo, Tn_lo, An_lo,""],
            "+1 Lognormal Standard Deviation (units)": [fn_hi, Tn_hi, An_hi,""]
        })

        csv_path = output_dir / f"{base_name}_summary.csv"
        with open(csv_path, "w", encoding="utf-8", newline="") as f:
            summary_df.to_csv(f, index=False)
            f.write("\n")
            f.write(f"The peak of the mean curve is at {fn:.3f} Hz with amplitude {An:.3f}.\n")

        # ✅ 그래프 저장 (PNG)
        fig, ax = hvsrpy.plot_single_panel_hvsr_curves(hvsr)

        # ✅ 기타 설정 (기존 코드 유지)
        ax.get_legend().remove()
        ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
        fig.savefig(output_dir / f"{base_name}_hvsr.png", bbox_inches="tight")
        plt.close(fig)

    except Exception as e:
        print(f"⚠️ 오류 발생: {base_name} - {e}")



▶ Processing: 453015908.0001.2025.03.23.03.34.18.000


,Exponentitated Lognormal Median (units),Lognormal Standard Deviation (log units),-1 Lognormal Standard Deviation (units),+1 Lognormal Standard Deviation (units)
"Resonant Site Frequency, fn (Hz)",0.965,0.899,0.393,2.373
"Resonant Site Period, Tn (s)",1.036,0.899,2.547,0.421
"Resonance Amplitude, An",13.285,0.539,7.747,22.780


⚠️ 오류 발생: 453015908.0001.2025.03.23.03.34.18.000 - 'peak_individual_valid_hvsr_curve'

▶ Processing: 453015908.0002.2025.03.23.04.40.34.000


,Exponentitated Lognormal Median (units),Lognormal Standard Deviation (log units),-1 Lognormal Standard Deviation (units),+1 Lognormal Standard Deviation (units)
"Resonant Site Frequency, fn (Hz)",1.332,0.955,0.513,3.463
"Resonant Site Period, Tn (s)",0.751,0.955,1.951,0.289
"Resonance Amplitude, An",12.470,0.616,6.732,23.099


⚠️ 오류 발생: 453015908.0002.2025.03.23.04.40.34.000 - 'peak_individual_valid_hvsr_curve'

▶ Processing: 453015908.0003.2025.03.23.05.43.22.000


...

C:\Users\USER\PycharmProjects\hvsrpy__\hvsrpy\postprocessing.py:348: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(**subplots_kwargs)


⚠️ 오류 발생: 453016751.0009.2025.04.02.00.34.46.000 - 'peak_individual_valid_hvsr_curve'

▶ Processing: 453016751.0010.2025.04.06.04.11.14.000


,Exponentitated Lognormal Median (units),Lognormal Standard Deviation (log units),-1 Lognormal Standard Deviation (units),+1 Lognormal Standard Deviation (units)
"Resonant Site Frequency, fn (Hz)",0.925,0.993,0.343,2.496
"Resonant Site Period, Tn (s)",1.081,0.993,2.918,0.401
"Resonance Amplitude, An",9.681,0.806,4.322,21.681


⚠️ 오류 발생: 453016751.0010.2025.04.06.04.11.14.000 - 'peak_individual_valid_hvsr_curve'

▶ Processing: 453016751.0011.2025.04.06.05.26.02.000


,Exponentitated Lognormal Median (units),Lognormal Standard Deviation (log units),-1 Lognormal Standard Deviation (units),+1 Lognormal Standard Deviation (units)
"Resonant Site Frequency, fn (Hz)",0.782,0.831,0.341,1.794
"Resonant Site Period, Tn (s)",1.279,0.831,2.937,0.557
"Resonance Amplitude, An",9.900,0.769,4.589,21.360
